In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(1234)

In [2]:
from requests import get as rget

In [3]:
from IPython.core.display import HTML

In [4]:
%%bash
if [ ! -f wikiFile.html ]
then
    wget https://en.wikipedia.org/wiki/List_of_S%26P_500_companies -O wikiFile.html
fi

In [5]:
wikifile = open("wikiFile.html", "r")

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(wikifile, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of S&amp;P 500 companies - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_S\u0026P_500_companies","wgTitle":"List of S\u0026P 500 companies","wgCurRevisionId":776766355,"wgRevisionId":776766355,"wgArticleId":2676045,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","S\u0026P Dow Jones Indices","Lists of companies","Standard \u0026 Poor's"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTab

In [7]:
tables = soup.select('body table')

In [8]:
sp_table = tables[0]

In [9]:
sp_trs = sp_table.select("tr")

In [10]:
sp500Dirs = {}
for row in sp_trs[1:]:
    changed = False
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    print("Company: ", companyName)
    tick = tds[0].find("a").text
    print("TICK: ", tick)
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("URL: ", url)
    break
    print("---")

Company:  3M
TICK:  MMM
URL:  https://www.bloomberg.com/quote/MMM:US


In [11]:
%%bash
if [ ! -f geckodriver ]
then
    if [ ! -f geckodriver.tar.gz ]
    then
        wget https://github.com/mozilla/geckodriver/releases/download/v0.16.0/geckodriver-v0.16.0-linux64.tar.gz -O geckodriver.tar.gz
    fi
    tar -zxvf geckodriver.tar.gz
fi

In [25]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

#Following are optional required
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

In [44]:
mydriver = webdriver.Firefox(executable_path='./geckodriver')

In [13]:
mydriver.get(url)
tmp = mydriver.find_element_by_css_selector(".management.show span.more")
tmp.click()
len(mydriver.find_elements_by_css_selector(".management.show li"))

5

In [14]:
import time

In [45]:
sp500Execs = {}
sp500Dirs  = {}
tick2comp = {}
for row in sp_trs[1:]:
    changed = False
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick]  = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName

Company:                   3M | URL: https://www.bloomberg.com/quote/MMM:US
No more Dirs
Company:  Abbott Laboratories | URL: https://www.bloomberg.com/quote/ABT:US
No more Dirs
Company:          AbbVie Inc. | URL: https://www.bloomberg.com/quote/ABBV:US
No more Dirs
Company:        Accenture plc | URL: https://www.bloomberg.com/quote/ACN:US
No more Dirs
Company:  Activision Blizzard | URL: https://www.bloomberg.com/quote/ATVI:US
No more Dirs
Company:    Acuity Brands Inc | URL: https://www.bloomberg.com/quote/AYI:US
No more Execs
Company:    Adobe Systems Inc | URL: https://www.bloomberg.com/quote/ADBE:US
No more Dirs
Company: Advanced Micro Devices Inc | URL: https://www.bloomberg.com/quote/AMD:US
No more Dirs
Company:   Advance Auto Parts | URL: https://www.bloomberg.com/quote/AAP:US
No more Dirs
Company:             AES Corp | URL: https://www.bloomberg.com/quote/AES:US
No more Dirs
Company:            Aetna Inc | URL: https://www.bloomberg.com/quote/AET:US
No more Dirs
Company: Af

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [55]:
sp_trs[66]

<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:BAX" rel="nofollow">BAX</a></td>
<td><a class="mw-redirect" href="/wiki/Baxter_International_Inc." title="Baxter International Inc.">Baxter International Inc.</a></td>
<td><a class="external text" href="http://www.sec.gov/cgi-bin/browse-edgar?CIK=BAX&amp;action=getcompany" rel="nofollow">reports</a></td>
<td>Health Care</td>
<td>Health Care Equipment</td>
<td><a href="/wiki/Deerfield,_Illinois" title="Deerfield, Illinois">Deerfield, Illinois</a></td>
<td>1972-09-30</td>
<td>0000010456</td>
</tr>

In [58]:
for row in sp_trs[67:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName

Company:     BB&T Corporation | URL: https://www.bloomberg.com/quote/BBT:US
No more Dirs
Company:     Becton Dickinson | URL: https://www.bloomberg.com/quote/BDX:US
No more Dirs
Company:    Bed Bath & Beyond | URL: https://www.bloomberg.com/quote/BBBY:US
No more Dirs
Company:   Berkshire Hathaway | URL: https://www.bloomberg.com/quote/BRK.B:US
No more Execs


IndexError: list index out of range

In [60]:
sp_trs[70]

<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:BRK.B" rel="nofollow">BRK.B</a></td>
<td><a href="/wiki/Berkshire_Hathaway" title="Berkshire Hathaway">Berkshire Hathaway</a></td>
<td><a class="external text" href="http://www.sec.gov/cgi-bin/browse-edgar?CIK=BRKB&amp;action=getcompany" rel="nofollow">reports</a></td>
<td>Financials</td>
<td>Multi-Sector Holdings</td>
<td><a href="/wiki/Omaha,_Nebraska" title="Omaha, Nebraska">Omaha, Nebraska</a></td>
<td>2010-02-16</td>
<td>0001067983</td>
</tr>

In [61]:
row = sp_trs[70]
tds = row.find_all("td")
companyName = tds[1].find("a").attrs["title"]
tick = "BRK"
url = "https://www.bloomberg.com/quote/" + tick + ":US"
print("Company: %20s | URL: %s" % (companyName, url))
mydriver.get(url)
time.sleep(0.5)
try:
    mydriver.find_element_by_css_selector(".management.show .more").click()
except:
    print("No more Execs")
time.sleep(0.5)

sp500Execs[tick] = set([
    ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
])
mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
time.sleep(0.5)

try:
    mydriver.find_element_by_css_selector(".management.show .more").click()
except:
    print("No more Dirs")
sp500Dirs[tick] = set([
    ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
])
sp500Dirs[tick].remove('')
tick2comp[tick] = companyName

Company:   Berkshire Hathaway | URL: https://www.bloomberg.com/quote/BRK:US
No more Dirs


In [66]:
for row in sp_trs[71:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName

Company:    Best Buy Co. Inc. | URL: https://www.bloomberg.com/quote/BBY:US
No more Dirs
Company:     BIOGEN IDEC Inc. | URL: https://www.bloomberg.com/quote/BIIB:US
No more Dirs
Company:            BlackRock | URL: https://www.bloomberg.com/quote/BLK:US
No more Dirs
Company:            Block H&R | URL: https://www.bloomberg.com/quote/HRB:US
No more Dirs
Company:       Boeing Company | URL: https://www.bloomberg.com/quote/BA:US
No more Dirs
Company:           BorgWarner | URL: https://www.bloomberg.com/quote/BWA:US
No more Dirs
Company:    Boston Properties | URL: https://www.bloomberg.com/quote/BXP:US
No more Dirs
Company:    Boston Scientific | URL: https://www.bloomberg.com/quote/BSX:US
No more Dirs
Company: Bristol-Myers Squibb | URL: https://www.bloomberg.com/quote/BMY:US
No more Dirs
Company:     Broadcom Limited | URL: https://www.bloomberg.com/quote/AVGO:US
No more Dirs
Company:   Brown-Forman Corp. | URL: https://www.bloomberg.com/quote/BF:US
No more Dirs
Company: C. H. Robins

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [72]:
sp_trs[91]

<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:CAT" rel="nofollow">CAT</a></td>
<td><a href="/wiki/Caterpillar_Inc." title="Caterpillar Inc.">Caterpillar Inc.</a></td>
<td><a class="external text" href="http://www.sec.gov/cgi-bin/browse-edgar?CIK=CAT&amp;action=getcompany" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Construction Machinery &amp; Heavy Trucks</td>
<td><a href="/wiki/Peoria,_Illinois" title="Peoria, Illinois">Peoria, Illinois</a></td>
<td></td>
<td>0000018230</td>
</tr>

In [73]:
counter = 91

In [74]:
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

Company:     Caterpillar Inc. | URL: https://www.bloomberg.com/quote/CAT:US
No more Dirs
Company:           CBRE Group | URL: https://www.bloomberg.com/quote/CBG:US
No more Dirs
Company:            CBS Corp. | URL: https://www.bloomberg.com/quote/CBS:US
No more Dirs
Company:        Celgene Corp. | URL: https://www.bloomberg.com/quote/CELG:US
No more Dirs
Company:  Centene Corporation | URL: https://www.bloomberg.com/quote/CNC:US
No more Dirs
Company:   CenterPoint Energy | URL: https://www.bloomberg.com/quote/CNP:US
No more Dirs
Company:      CenturyLink Inc | URL: https://www.bloomberg.com/quote/CTL:US
No more Dirs
Company:               Cerner | URL: https://www.bloomberg.com/quote/CERN:US
No more Dirs
Company: CF Industries Holdings Inc | URL: https://www.bloomberg.com/quote/CF:US
No more Dirs
Company: Charles Schwab Corporation | URL: https://www.bloomberg.com/quote/SCHW:US
No more Dirs
Company: Charter Communications | URL: https://www.bloomberg.com/quote/CHTR:US
No more Dirs
Comp

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [75]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

142
Company:    Delphi Automotive | URL: https://www.bloomberg.com/quote/DLPH:US
No more Dirs
Company:      Delta Air Lines | URL: https://www.bloomberg.com/quote/DAL:US
No more Dirs
Company:      Dentsply Sirona | URL: https://www.bloomberg.com/quote/XRAY:US
No more Dirs
Company:   Devon Energy Corp. | URL: https://www.bloomberg.com/quote/DVN:US
No more Dirs
Company: Digital Realty Trust | URL: https://www.bloomberg.com/quote/DLR:US
No more Dirs
Company: Discover Financial Services | URL: https://www.bloomberg.com/quote/DFS:US
No more Dirs
Company: Discovery Communications-A | URL: https://www.bloomberg.com/quote/DISCA:US
No more Dirs
Company: Discovery Communications-C | URL: https://www.bloomberg.com/quote/DISCK:US
No more Dirs
Company:         Dish Network | URL: https://www.bloomberg.com/quote/DISH:US
No more Dirs
Company:       Dollar General | URL: https://www.bloomberg.com/quote/DG:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [76]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

151
Company:       Dollar General | URL: https://www.bloomberg.com/quote/DG:US
No more Dirs
Company:          Dollar Tree | URL: https://www.bloomberg.com/quote/DLTR:US
No more Dirs
Company:   Dominion Resources | URL: https://www.bloomberg.com/quote/D:US
No more Dirs
Company:          Dover Corp. | URL: https://www.bloomberg.com/quote/DOV:US
No more Dirs
Company:         Dow Chemical | URL: https://www.bloomberg.com/quote/DOW:US
No more Dirs
Company: Dr Pepper Snapple Group | URL: https://www.bloomberg.com/quote/DPS:US
No more Dirs
Company:       DTE Energy Co. | URL: https://www.bloomberg.com/quote/DTE:US
No more Dirs
Company:       Du Pont (E.I.) | URL: https://www.bloomberg.com/quote/DD:US
No more Dirs
Company:          Duke Energy | URL: https://www.bloomberg.com/quote/DUK:US
No more Dirs
Company:       DXC Technology | URL: https://www.bloomberg.com/quote/DXC:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [77]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

160
Company:       DXC Technology | URL: https://www.bloomberg.com/quote/DXC:US
No more Dirs
Company:              E*Trade | URL: https://www.bloomberg.com/quote/ETFC:US
No more Dirs
Company:     Eastman Chemical | URL: https://www.bloomberg.com/quote/EMN:US
No more Dirs
Company:    Eaton Corporation | URL: https://www.bloomberg.com/quote/ETN:US
No more Dirs
Company:            EBay Inc. | URL: https://www.bloomberg.com/quote/EBAY:US
No more Dirs
Company:          Ecolab Inc. | URL: https://www.bloomberg.com/quote/ECL:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [78]:
mydriver.close()
mydriver.get("http://aaron.niskin.org")

In [79]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

165
Company:          Ecolab Inc. | URL: https://www.bloomberg.com/quote/ECL:US
No more Dirs
Company:         Edison Int'l | URL: https://www.bloomberg.com/quote/EIX:US
No more Dirs
Company: Edwards Lifesciences | URL: https://www.bloomberg.com/quote/EW:US
No more Dirs
Company:      Electronic Arts | URL: https://www.bloomberg.com/quote/EA:US
No more Dirs
Company: Emerson Electric Company | URL: https://www.bloomberg.com/quote/EMR:US
No more Dirs
Company:        Entergy Corp. | URL: https://www.bloomberg.com/quote/ETR:US
No more Dirs
Company:  Envision Healthcare | URL: https://www.bloomberg.com/quote/EVHC:US
No more Dirs
Company:        EOG Resources | URL: https://www.bloomberg.com/quote/EOG:US
No more Dirs
Company:      EQT Corporation | URL: https://www.bloomberg.com/quote/EQT:US
No more Dirs
Company:         Equifax Inc. | URL: https://www.bloomberg.com/quote/EFX:US
No more Dirs
Company:              Equinix | URL: https://www.bloomberg.com/quote/EQIX:US
No more Dirs
Company:   Eq

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [80]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

210
Company:     General Electric | URL: https://www.bloomberg.com/quote/GE:US
No more Dirs
Company: General Growth Properties Inc. | URL: https://www.bloomberg.com/quote/GGP:US
No more Dirs
Company:        General Mills | URL: https://www.bloomberg.com/quote/GIS:US
No more Dirs
Company:       General Motors | URL: https://www.bloomberg.com/quote/GM:US
No more Dirs
Company:        Genuine Parts | URL: https://www.bloomberg.com/quote/GPC:US
No more Dirs
Company:      Gilead Sciences | URL: https://www.bloomberg.com/quote/GILD:US
No more Dirs
Company: Heartland Payment Systems | URL: https://www.bloomberg.com/quote/GPN:US
No more Dirs
Company:  Goldman Sachs Group | URL: https://www.bloomberg.com/quote/GS:US
No more Dirs
Company: Goodyear Tire & Rubber | URL: https://www.bloomberg.com/quote/GT:US
No more Dirs
Company: Grainger (W.W.) Inc. | URL: https://www.bloomberg.com/quote/GWW:US
No more Dirs
Company:      Halliburton Co. | URL: https://www.bloomberg.com/quote/HAL:US
No more Dirs
Com

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [81]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    time.sleep(0.5)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    time.sleep(0.5)
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

221
Company:      Hanesbrands Inc | URL: https://www.bloomberg.com/quote/HBI:US
No more Dirs
Company:      Harley-Davidson | URL: https://www.bloomberg.com/quote/HOG:US
No more Dirs
Company:   Harris Corporation | URL: https://www.bloomberg.com/quote/HRS:US
No more Dirs
Company: Hartford Financial Svc.Gp. | URL: https://www.bloomberg.com/quote/HIG:US
No more Dirs
Company:          Hasbro Inc. | URL: https://www.bloomberg.com/quote/HAS:US
No more Dirs
Company:         HCA Holdings | URL: https://www.bloomberg.com/quote/HCA:US
No more Dirs
Company:             HCP Inc. | URL: https://www.bloomberg.com/quote/HCP:US
No more Dirs
Company:    Helmerich & Payne | URL: https://www.bloomberg.com/quote/HP:US
No more Dirs
Company:         Henry Schein | URL: https://www.bloomberg.com/quote/HSIC:US
No more Dirs
Company:     Hess Corporation | URL: https://www.bloomberg.com/quote/HES:US
No more Dirs
Company: Hewlett Packard Enterprise | URL: https://www.bloomberg.com/quote/HPE:US
No more Dirs
Compa

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [82]:
counter

264

In [83]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

264
Company:              KeyCorp | URL: https://www.bloomberg.com/quote/KEY:US
No more Dirs
Company:       Kimberly-Clark | URL: https://www.bloomberg.com/quote/KMB:US
No more Dirs
Company:         Kimco Realty | URL: https://www.bloomberg.com/quote/KIM:US
No more Dirs
Company:        Kinder Morgan | URL: https://www.bloomberg.com/quote/KMI:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [84]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

267
Company:        Kinder Morgan | URL: https://www.bloomberg.com/quote/KMI:US
No more Dirs
Company:     KLA-Tencor Corp. | URL: https://www.bloomberg.com/quote/KLAC:US
No more Dirs
Company:         Kohl's Corp. | URL: https://www.bloomberg.com/quote/KSS:US
No more Dirs
Company:          Kraft Heinz | URL: https://www.bloomberg.com/quote/KHC:US
No more Dirs
Company:           Kroger Co. | URL: https://www.bloomberg.com/quote/KR:US
No more Dirs
Company:        L Brands Inc. | URL: https://www.bloomberg.com/quote/LB:US
No more Dirs
Company: L-3 Communications Holdings | URL: https://www.bloomberg.com/quote/LLL:US
No more Dirs
Company: Laboratory Corp. of America Holding | URL: https://www.bloomberg.com/quote/LH:US
No more Dirs
Company:         Lam Research | URL: https://www.bloomberg.com/quote/LRCX:US
No more Dirs
Company:      Leggett & Platt | URL: https://www.bloomberg.com/quote/LEG:US
No more Dirs
Company:         Lennar Corp. | URL: https://www.bloomberg.com/quote/LEN:US
No more D

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [85]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

315
Company:         Monsanto Co. | URL: https://www.bloomberg.com/quote/MON:US
No more Dirs
Company:     Monster Beverage | URL: https://www.bloomberg.com/quote/MNST:US
No more Execs
Company:         Moody's Corp | URL: https://www.bloomberg.com/quote/MCO:US
No more Dirs
Company:       Morgan Stanley | URL: https://www.bloomberg.com/quote/MS:US
No more Dirs
Company:   The Mosaic Company | URL: https://www.bloomberg.com/quote/MOS:US
No more Dirs
Company: Motorola Solutions Inc. | URL: https://www.bloomberg.com/quote/MSI:US
No more Dirs
Company:           Murphy Oil | URL: https://www.bloomberg.com/quote/MUR:US
No more Dirs
Company:           Mylan N.V. | URL: https://www.bloomberg.com/quote/MYL:US
No more Dirs
Company:         Nasdaq, Inc. | URL: https://www.bloomberg.com/quote/NDAQ:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [86]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

323
Company:         Nasdaq, Inc. | URL: https://www.bloomberg.com/quote/NDAQ:US
No more Dirs
Company: National Oilwell Varco Inc. | URL: https://www.bloomberg.com/quote/NOV:US
No more Dirs
Company:              Navient | URL: https://www.bloomberg.com/quote/NAVI:US
No more Dirs
Company:               NetApp | URL: https://www.bloomberg.com/quote/NTAP:US
No more Dirs
Company:         Netflix Inc. | URL: https://www.bloomberg.com/quote/NFLX:US
No more Dirs
Company:        Newell Brands | URL: https://www.bloomberg.com/quote/NWL:US
No more Dirs
Company: Newfield Exploration Co | URL: https://www.bloomberg.com/quote/NFX:US
No more Dirs
Company: Newmont Mining Corp. (Hldg. Co.) | URL: https://www.bloomberg.com/quote/NEM:US
No more Dirs
Company:   News Corp. Class A | URL: https://www.bloomberg.com/quote/NWSA:US
No more Dirs
Company:   News Corp. Class B | URL: https://www.bloomberg.com/quote/NWS:US
No more Dirs
Company:       NextEra Energy | URL: https://www.bloomberg.com/quote/NEE:US
No 

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [87]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

338
Company:            Nordstrom | URL: https://www.bloomberg.com/quote/JWN:US
No more Dirs
Company: Norfolk Southern Corp. | URL: https://www.bloomberg.com/quote/NSC:US
No more Dirs
Company: Northern Trust Corp. | URL: https://www.bloomberg.com/quote/NTRS:US
No more Dirs
Company: Northrop Grumman Corp. | URL: https://www.bloomberg.com/quote/NOC:US
No more Dirs
Company:           NRG Energy | URL: https://www.bloomberg.com/quote/NRG:US
No more Dirs
Company:          Nucor Corp. | URL: https://www.bloomberg.com/quote/NUE:US
No more Dirs
Company:   Nvidia Corporation | URL: https://www.bloomberg.com/quote/NVDA:US
No more Dirs
Company:  O'Reilly Automotive | URL: https://www.bloomberg.com/quote/ORLY:US
No more Dirs
Company: Occidental Petroleum | URL: https://www.bloomberg.com/quote/OXY:US
No more Dirs
Company:        Omnicom Group | URL: https://www.bloomberg.com/quote/OMC:US
No more Dirs
Company:                ONEOK | URL: https://www.bloomberg.com/quote/OKE:US
No more Dirs
Company:  

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [88]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

363
Company:          Phillips 66 | URL: https://www.bloomberg.com/quote/PSX:US
No more Dirs
Company: Arizona Public Service | URL: https://www.bloomberg.com/quote/PNW:US
No more Dirs
Company: Pioneer Natural Resources | URL: https://www.bloomberg.com/quote/PXD:US
No more Dirs
Company: PNC Financial Services | URL: https://www.bloomberg.com/quote/PNC:US
No more Dirs
Company: Polo Ralph Lauren Corp. | URL: https://www.bloomberg.com/quote/RL:US
No more Dirs
Company:       PPG Industries | URL: https://www.bloomberg.com/quote/PPG:US
No more Dirs
Company:            PPL Corp. | URL: https://www.bloomberg.com/quote/PPL:US
No more Dirs
Company:         Praxair Inc. | URL: https://www.bloomberg.com/quote/PX:US
No more Dirs
Company:  The Priceline Group | URL: https://www.bloomberg.com/quote/PCLN:US
No more Execs
Company: Principal Financial Group | URL: https://www.bloomberg.com/quote/PFG:US
No more Dirs
Company:     Procter & Gamble | URL: https://www.bloomberg.com/quote/PG:US
No more Dirs
C

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [89]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

421
Company:   State Street Corp. | URL: https://www.bloomberg.com/quote/STT:US
No more Dirs
Company:       Stericycle Inc | URL: https://www.bloomberg.com/quote/SRCL:US
No more Dirs
Company:        Stryker Corp. | URL: https://www.bloomberg.com/quote/SYK:US
No more Dirs
Company:       SunTrust Banks | URL: https://www.bloomberg.com/quote/STI:US
No more Dirs
Company:       Symantec Corp. | URL: https://www.bloomberg.com/quote/SYMC:US
No more Dirs
Company:  Synchrony Financial | URL: https://www.bloomberg.com/quote/SYF:US
No more Dirs
Company:        Synopsys Inc. | URL: https://www.bloomberg.com/quote/SNPS:US
No more Dirs
Company:          Sysco Corp. | URL: https://www.bloomberg.com/quote/SYY:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [90]:
print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1

428
Company:          Sysco Corp. | URL: https://www.bloomberg.com/quote/SYY:US
No more Dirs
Company:  T. Rowe Price Group | URL: https://www.bloomberg.com/quote/TROW:US
No more Dirs
Company:         Target Corp. | URL: https://www.bloomberg.com/quote/TGT:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [91]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

430
Company:         Target Corp. | URL: https://www.bloomberg.com/quote/TGT:US
No more Dirs
Company:      TE Connectivity | URL: https://www.bloomberg.com/quote/TEL:US
No more Dirs
Company:          Tegna, Inc. | URL: https://www.bloomberg.com/quote/TGNA:US
No more Dirs
Company:       Teradata Corp. | URL: https://www.bloomberg.com/quote/TDC:US
No more Dirs
Company: Tesoro Petroleum Co. | URL: https://www.bloomberg.com/quote/TSO:US
No more Dirs
Company:    Texas Instruments | URL: https://www.bloomberg.com/quote/TXN:US
No more Dirs
Company:         Textron Inc. | URL: https://www.bloomberg.com/quote/TXT:US
No more Dirs
Company: The Cooper Companies | URL: https://www.bloomberg.com/quote/COO:US
No more Dirs
Company:  The Hershey Company | URL: https://www.bloomberg.com/quote/HSY:US
No more Dirs
Company: The Travelers Companies Inc. | URL: https://www.bloomberg.com/quote/TRV:US
No more Dirs
Company: Thermo Fisher Scientific | URL: https://www.bloomberg.com/quote/TMO:US
No more Dirs
Comp

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [92]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

471
Company:     Verisk Analytics | URL: https://www.bloomberg.com/quote/VRSK:US
No more Dirs
Company: Verizon Communications | URL: https://www.bloomberg.com/quote/VZ:US
No more Dirs
Company: Vertex Pharmaceuticals Inc | URL: https://www.bloomberg.com/quote/VRTX:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [93]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

473
Company: Vertex Pharmaceuticals Inc | URL: https://www.bloomberg.com/quote/VRTX:US
No more Dirs
Company:          Viacom Inc. | URL: https://www.bloomberg.com/quote/VIAB:US
No more Dirs
Company:            Visa Inc. | URL: https://www.bloomberg.com/quote/V:US
No more Dirs
Company: Vornado Realty Trust | URL: https://www.bloomberg.com/quote/VNO:US
No more Dirs
Company:     Vulcan Materials | URL: https://www.bloomberg.com/quote/VMC:US
No more Dirs
Company:      Wal-Mart Stores | URL: https://www.bloomberg.com/quote/WMT:US
No more Dirs
Company: Walgreens Boots Alliance | URL: https://www.bloomberg.com/quote/WBA:US
No more Dirs
Company: The Walt Disney Company | URL: https://www.bloomberg.com/quote/DIS:US
No more Dirs
Company: Waste Management Inc. | URL: https://www.bloomberg.com/quote/WM:US
No more Dirs
Company:   Waters Corporation | URL: https://www.bloomberg.com/quote/WAT:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [94]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

482
Company:   Waters Corporation | URL: https://www.bloomberg.com/quote/WAT:US
No more Dirs
Company: Wec Energy Group Inc | URL: https://www.bloomberg.com/quote/WEC:US
No more Dirs
Company:          Wells Fargo | URL: https://www.bloomberg.com/quote/WFC:US
No more Dirs
Company:       Welltower Inc. | URL: https://www.bloomberg.com/quote/HCN:US
No more Dirs
Company:      Western Digital | URL: https://www.bloomberg.com/quote/WDC:US
No more Dirs
Company:     Western Union Co | URL: https://www.bloomberg.com/quote/WU:US
No more Dirs
Company:     WestRock Company | URL: https://www.bloomberg.com/quote/WRK:US
No more Dirs
Company:   Weyerhaeuser Corp. | URL: https://www.bloomberg.com/quote/WY:US
No more Dirs
Company:      Whirlpool Corp. | URL: https://www.bloomberg.com/quote/WHR:US
No more Dirs
Company:   Whole Foods Market | URL: https://www.bloomberg.com/quote/WFM:US
No more Dirs
Company:        Williams Cos. | URL: https://www.bloomberg.com/quote/WMB:US
No more Dirs
Company: Willis Tow

StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [95]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

497
Company:          Xerox Corp. | URL: https://www.bloomberg.com/quote/XRX:US
No more Dirs
Company:           Xilinx Inc | URL: https://www.bloomberg.com/quote/XLNX:US
No more Dirs
Company:           XL Capital | URL: https://www.bloomberg.com/quote/XL:US
No more Dirs
Company:           Xylem Inc. | URL: https://www.bloomberg.com/quote/XYL:US
No more Dirs
Company:           Yahoo Inc. | URL: https://www.bloomberg.com/quote/YHOO:US
No more Dirs
Company:      Yum! Brands Inc | URL: https://www.bloomberg.com/quote/YUM:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [96]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

502
Company:      Yum! Brands Inc | URL: https://www.bloomberg.com/quote/YUM:US
No more Dirs
Company:      Zimmer Holdings | URL: https://www.bloomberg.com/quote/ZBH:US


StaleElementReferenceException: Message: The element reference of <li> stale: either the element is no longer attached to the DOM or the page has been refreshed


In [97]:
mydriver.close()

print(counter)
for row in sp_trs[counter:]:
    tds = row.find_all("td")
    companyName = tds[1].find("a").attrs["title"]
    tick = tds[0].find("a").text.split(".")[0]
    url = "https://www.bloomberg.com/quote/" + tick + ":US"
    print("Company: %20s | URL: %s" % (companyName, url))
    mydriver.get(url)
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Execs")
    time.sleep(0.5)
    
    sp500Execs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    mydriver.find_elements_by_css_selector(".management-nav a")[1].click()
    time.sleep(0.5)
    
    try:
        mydriver.find_element_by_css_selector(".management.show .more").click()
    except:
        print("No more Dirs")
    time.sleep(0.5)
    sp500Dirs[tick] = set([
        ls.text.split("\n")[0] for ls in mydriver.find_elements_by_css_selector(".management.show li")
    ])
    sp500Dirs[tick].remove('')
    tick2comp[tick] = companyName
    counter += 1
    

503
Company:      Zimmer Holdings | URL: https://www.bloomberg.com/quote/ZBH:US
No more Dirs
Company:        Zions Bancorp | URL: https://www.bloomberg.com/quote/ZION:US
No more Dirs
Company:               Zoetis | URL: https://www.bloomberg.com/quote/ZTS:US
No more Dirs


In [99]:
with open("sp500Dirs.csv", "w") as f:
    for key in sp500Dirs.keys():
        f.write(key + "," + ",".join(sp500Dirs[key]) + "\n")

232

209

170

206

159

...

192

204

251

177

183

In [100]:
with open("sp500Execs.csv", "w") as f:
    for key in sp500Execs.keys():
        f.write(key + "," + ",".join(sp500Execs[key]) + "\n");

291

359

181

314

169

...

280

122

169

80

187

In [102]:
with open("tick2comp.csv", "w") as f:
    for key in tick2comp.keys():
        f.write(key + "," + tick2comp[key] + "\n")

7

24

17

18

25

...

16

20

20

19

11